In [40]:
import pandas as pd
import numpy as np
import glob
import os

In [15]:
# Define the directory path where the files are located
directory_path = "data"

# List all .csv and .xls* files in the directory
csv_files = glob.glob(os.path.join(directory_path, "*.csv"))
xls_files = glob.glob(os.path.join(directory_path, "*.xls*"))  # This will match .xls and .xlsx

# Combine the lists of files
all_files = csv_files + xls_files

# Display the list of files
for file in all_files:
    print(file)

data/29.03.2024, PV 22.03.2024.csv


In [37]:
file = "data/29.03.2024, PV 22.03.2024.csv"

df = pd.read_csv(file, sep=';', decimal=',', skiprows=2)


df

,Datum,Uhrzeit,msek,WG MS Impulse,WG MS Beiwert,TiefeMesssonde,WG MS Ausgangswert,DS1 mA,DS2 mA,IDM mA,...,Druck EWS (bar),G-Wasser DS1,G-Wasser DS2,Schutzverrohr. (m),MS BL-Fuss,WG VL Impulse,WG VL Beiwert,TiefeV-Leitung,WG VL Ausgangswert,MS BL-Fuss.1
0,22.03.2024,09:51:51,769,NaN,1319.756,0,0,6.127,6.460,3.999,...,0,0,0.000000,0,Nein,NaN,1319.756,0,0,Nein
1,22.03.2024,09:51:52,253,NaN,1319.756,0,0,6.127,6.460,3.999,...,0,0,0.000000,0,Nein,NaN,1319.756,0,0,Nein
2,22.03.2024,09:51:52,346,NaN,1319.756,0,0,6.127,6.460,3.999,...,0,0,0.000000,0,Nein,NaN,1319.756,0,0,Nein
3,22.03.2024,09:51:52,549,NaN,1319.756,0,0,6.124,6.459,3.999,...,0,-1,0.000000,0,Nein,NaN,1319.756,0,0,Nein
4,22.03.2024,09:51:52,752,NaN,1319.756,0,0,6.124,6.457,3.999,...,0,-1,0.000000,0,Nein,NaN,1319.756,0,0,Nein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45822,22.03.2024,15:39:53,219,NaN,1319.756,0,0,6.143,7.486,0.000,...,0,0,-12.793248,0,Nein,NaN,1319.756,0,0,Nein
45823,22.03.2024,15:39:53,905,NaN,1319.756,0,0,6.141,7.487,0.000,...,0,0,-12.805728,0,Nein,NaN,1319.756,0,0,Nein
45824,22.03.2024,15:39:54,545,NaN,1319.756,0,0,6.141,7.481,0.000,...,0,0,-12.730848,0,Nein,NaN,1319.756,0,0,Nein
45825,22.03.2024,15:39:55,200,NaN,1319.756,0,0,6.135,7.486,0.000,...,0,0,-12.793248,0,Nein,NaN,1319.756,0,0,Nein


In [38]:
# Combine 'Datum' and 'Uhrzeit' into a single datetime column and set as index
df['DateTime'] = pd.to_datetime(df['Datum'] + ' ' + df['Uhrzeit'], dayfirst=True)
df.set_index('DateTime', inplace=True)

# Drop the original 'Datum' and 'Uhrzeit' columns as they are no longer needed
df.drop(columns=['Datum', 'Uhrzeit'], inplace=True)

In [42]:

# Exclude non-numeric columns from the normalization process
numeric_df = df.select_dtypes(include=[np.number])  # Use pd.np.number to select numeric columns
non_numeric_df = df.select_dtypes(exclude=[np.number])


# Define the normalization function with updated interval notation
def normalize_data(df, interval='s'):
    """Normalizes numeric data by calculating the mean over the specified interval."""
    return df.resample(interval).mean()

normalized_df_1s = normalize_data(numeric_df, '1s')  # Example for 1-second intervals

# Step 3: Resample non-numeric columns
# For non-numeric data, we might want to take the first value in each interval as an example
# Adjust the method according to your specific needs
resampled_non_numeric_df_1s = non_numeric_df.resample('1s').first()

# Step 4: Combine the normalized numeric DataFrame with the resampled non-numeric DataFrame
final_normalized_df_1s = pd.concat([normalized_df_1s, resampled_non_numeric_df_1s], axis=1)
# Note: If you need to handle the non-numeric columns differently, additional logic will be required.


In [43]:
normalized_df_1s

,msek,WG MS Impulse,WG MS Beiwert,TiefeMesssonde,WG MS Ausgangswert,DS1 mA,DS2 mA,IDM mA,DDS1 mA,DSEWS mA,...,Gesamtmenge Liter,DEWS Beiwert,Druck EWS (bar),G-Wasser DS1,G-Wasser DS2,Schutzverrohr. (m),WG VL Impulse,WG VL Beiwert,TiefeV-Leitung,WG VL Ausgangswert
DateTime,,,,,,,,,,,,,,,,,,,,,
2024-03-22 09:51:51,769.0,NaN,1319.756,0.0,0.0,6.1270,6.4600,3.999,4.0170,0.0,...,0.00,1.6,0.0,0.0,0.000000,0.0,NaN,1319.756,0.0,0.0
2024-03-22 09:51:52,571.0,NaN,1319.756,0.0,0.0,6.1252,6.4586,3.999,4.0170,0.0,...,0.00,1.6,0.0,-0.6,0.000000,0.0,NaN,1319.756,0.0,0.0
2024-03-22 09:51:53,563.0,NaN,1319.756,0.0,0.0,6.1218,6.4544,3.999,4.0170,0.0,...,0.00,1.6,0.0,-1.0,0.000000,0.0,NaN,1319.756,0.0,0.0
2024-03-22 09:51:54,577.0,NaN,1319.756,0.0,0.0,6.1194,6.4572,3.999,4.0174,0.0,...,0.00,1.6,0.0,-1.0,0.000000,0.0,NaN,1319.756,0.0,0.0
2024-03-22 09:51:55,591.0,NaN,1319.756,0.0,0.0,6.1216,6.4588,3.999,4.0170,0.0,...,0.00,1.6,0.0,-1.0,0.000000,0.0,NaN,1319.756,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-22 15:39:51,456.0,NaN,1319.756,0.0,0.0,6.1445,7.4810,0.000,0.0000,0.0,...,34937.96,1.6,0.0,0.0,-12.730848,0.0,NaN,1319.756,0.0,0.0
2024-03-22 15:39:52,501.0,NaN,1319.756,0.0,0.0,6.1430,7.4810,0.000,0.0000,0.0,...,34937.96,1.6,0.0,0.0,-12.730848,0.0,NaN,1319.756,0.0,0.0
2024-03-22 15:39:53,562.0,NaN,1319.756,0.0,0.0,6.1420,7.4865,0.000,0.0000,0.0,...,34937.96,1.6,0.0,0.0,-12.799488,0.0,NaN,1319.756,0.0,0.0


In [35]:
for col in df.columns:
    if df[col].dtype == 'object':
        print(f'Column "{col}" contains non-numeric values.')

Column "MS BL-Fuss" contains non-numeric values.
Column "MS BL-Fuss.1" contains non-numeric values.


In [30]:
normalized_df_1s = normalize_data(df, '1S')  # Normalize over each second
normalized_df_2s = normalize_data(df, '2S')  # Normalize over two seconds
normalized_df_4s = normalize_data(df, '4S')  # Normalize over four seconds

/var/folders/9x/xg2m4d210h168qwt8x3979580000gn/T/ipykernel_96459/3673084513.py:13: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  return df.resample(interval).mean()


TypeError: agg function failed [how->mean,dtype->object]